
<div class="text-center">
<h1 class="display-1" >Data Mining </h1>
<h3> Homework 2</h3>
<h3> Di Santo Nicola - 1853049</h3>

</div>

# Exercise 1

## Utilities
In this section we have all the import of useful libraries like: 
<ul>
    <li>nltk to preporcess documents and query as explained in the hw.pdf file </li>
    <li>pandas to easly load and manipulate data</li> 
     <li>unicodedata is exploited to remove accents</li>
</ul>
And some useful function:<ul><li> strip_accents() function that exploit unicode normalization to remove accents</li><li> process_query() function that normalize the query entered by the user and returns the vector form</li><li>perform_query() function, that takes as input the inverted index, the query in its vector form, a list containing the length of all the documents in the system and the number k of top document to be retrived and return the k tuples containing (doc_id,score) representing the top-k documents according to cosine similarity and tfidf weight</li><li>get_document_length() funciton returns the length of the vectorized form of a document.</li>
    
### Perform Query
The perform_query() function perform the fast cosine similarity algorithm:
<img src="fcs.JPG"></img>


In [1]:
#utilities
import unicodedata, nltk, re, heapq, math,ast
nltk.download('stopwords') #comment if already present and warning bothers you
nltk.download('punkt') #comment if already present and warning bothers you
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
import pandas as pd

#Remove the accents from text exploiting unicode normalization
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)


'''

'''
def process_query(query):
    stop_words_it = set(stopwords.words('italian'))
    stemmer = SnowballStemmer("italian")  
    
    query = re.sub("`|'|!|\||\\|#|\$|%|\^|&|\*|\(|\)|-|_|\[|\]|\{|\}|\"|\'|\+|:|;|<|>|\?|/|`|~|@|=|,|\.", ' ', strip_accents(query).lower())
    query_tokens = word_tokenize(query) 
    #removing italian stopwords
    filtered_query = [w for w in query_tokens if not w in stop_words_it]
    query_vect = set()
    for word in filtered_query:
        #stemming tokens
        sword = stemmer.stem(word)
        query_vect.add(sword)
    return query_vect

# sorted_idx: pandas dataframe
# query_vect is a list of terms appearing in the query already preprocessed
# len_d: list containing the lenght of each document, it's computed as initialization step of the query system
#        position i represent document i
# k: number of top document to be returned from query
def perform_query(sorted_idx,query_vect,len_d, k):
    scores = {}
    for query_term in query_vect:
       
        term_row = sorted_idx.loc[sorted_idx['Term'] == query_term]
        if not term_row.empty:
            
            term_idf = math.log(term_row['idf'],10)
            term_postings = ast.literal_eval(term_row.postings.values[0]) #safely evaluate and convert from string to list data structure

            for document,tf in term_postings:
                if document not in scores:
                    scores[document]= float(tf)*term_idf
                else:
                    scores[document]= scores[document] + tf*term_idf

    for document, weight in scores.items():
        scores[document]= weight/len_d[document]
    
   
    sorted_scores= sorted(scores.items(),key=lambda tup: tup[1], reverse=True) 
    #sort scores by weight and return top k
    top_k = []
    if k > len(sorted_scores):
        k = len(sorted_scores)
        
    for i in range(0,k):
        top_k.append(sorted_scores[i])
    return top_k

def get_document_length(document):
    
    stop_words_it = set(stopwords.words('italian'))
    stemmer = SnowballStemmer("italian")  
    
    document = re.sub("`|'|!|\||\\|#|\$|%|\^|&|\*|\(|\)|-|_|\[|\]|\{|\}|\"|\'|\+|:|;|<|>|\?|/|`|~|@|=|,|\.", ' ', strip_accents(document).lower())
    document_tokens = word_tokenize(document) 
    #removing italian stopwords
    filtered_doc = [w for w in document_tokens if not w in stop_words_it]
    document_vect = set()
    for word in filtered_doc:
        #stemming tokens
        sword = stemmer.stem(word)
        document_vect.add(sword)
    return len(document_vect)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nds\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nds\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading the dataset and showing an overview

In [2]:
dataset=pd.read_csv("dataset_rent_rome_kijiji.tsv", sep="\t")
dataset.head()

,Title,Short Description,Location,Price (Euro),Timestamp,Url Adv
0,Studio accessoriato vicino metro A Furio Camillo,Affitto studio a professionisti preferibilment...,Roma,450,"12 ottobre, 11:32",https://www.kijiji.it/annunci/affitto/roma-ann...
1,"Negozio 169Mq per laboratorio, ufficio, studio...","Privato affitta negozio 169 mq, al piano terra...",Prenestino / Casilino,1.700,"12 ottobre, 08:45",https://www.kijiji.it/annunci/affitto/roma-ann...
2,Negozio in tiburtina centro,Negozio c/1 roma tiburtina centro via eugenio ...,Tiburtino / Collatino,6.000,"17 October, 21:20",https://www.kijiji.it/annunci/affitto/roma-ann...
3,Studio medico via anapo parco nemorense,"Studio medico avviato, composto da tre studi c...",Trieste / Somalia / Salario,200,"17 October, 20:22",https://www.kijiji.it/annunci/affitto/roma-ann...
4,Cerco: Appartamento per donna lavoratrice refe...,"Donna lavoratrice, non residente, con reddito ...",Roma,Contatta l'utente,"17 October, 19:39",https://www.kijiji.it/annunci/affitto/roma-ann...


In [3]:
dataset.tail()

,Title,Short Description,Location,Price (Euro),Timestamp,Url Adv
2622,Eur corazzieri,EUR CORAZZIERI in una bella palazzina disponia...,Roma,1.250,"13 febbraio, 23:47",https://www.kijiji.it/annunci/affitto/roma-ann...
2623,Re di roma,"RE DI ROMA, a pochi passi dalla metro e precis...",Roma,1.200,"13 febbraio, 23:47",https://www.kijiji.it/annunci/affitto/roma-ann...
2624,Ponte di nona,PONTE DI NONA ATTICO in complesso residenziale...,Roma,800,"13 febbraio, 23:47",https://www.kijiji.it/annunci/affitto/roma-ann...
2625,Trieste,TRIESTE corso PANORAMICISSIMO! fronte mausoleo...,Roma,800,"13 febbraio, 23:47",https://www.kijiji.it/annunci/affitto/roma-ann...
2626,Marconi,MARCONI adiac. lungotevere Pietra Papa disponi...,Roma,700,"13 febbraio, 23:47",https://www.kijiji.it/annunci/affitto/roma-ann...


## Preprocessing 

Recall, from description of the homewrok attached, that a document is a string obtained concatenating the fields <i>Title</i> and <i>Short Description</i>. As preprocesing step we first assign to each document a document id exploiting the index of the dataframe, then we drop all the columns that are not usfull for the construction of the index. Then the subsequent steps consist into removing special characters from a document and case-folding them. After this step we iterate over the Dataframe to tokenize and stem as well as build the inverted index as a nested dictionary in the form {term:{doc_id:term_freq}}. The dictionary will the contain as key the normalized term, as value a dictionary. The inner dictionary has as key the documenti id and as value the term frequency of the term in that document.<br>After the index is contructed it is written on a file called <i>inverted_index.txt</i>.<br>Detail of the index construction are in the solution description attached. 

In [4]:
#dropping unusefull informations

# 1. create doc_ids: 
#    we will use the index of Dataframe
dataset['doc_id']=dataset.index

dataset = dataset.drop(columns=['Location','Price (Euro)','Timestamp','Url Adv'])
dataset.head()
print('ok...')

ok...


In [5]:
# 2. for each row preprocess:
#    1 normalize text removing accents, dots, commas, columns, ;, casefolding, - , _ , \ | / ? < [ { } ] > 
#    2 Tokenize splitting on space
#    3 Stemming   
term_dict = {}
stop_words_it = set(stopwords.words('italian'))
stemmer = SnowballStemmer("italian")  
doc_nr = 2626

for index, row in dataset.iterrows():
    dataset.at[index,'Title'] = re.sub("`|'|!|\||\\|#|\$|%|\^|&|\*|\(|\)|-|_|\[|\]|\{|\}|\"|\'|\+|:|;|<|>|\?|/|`|~|@|=|,|\.", ' ', strip_accents(row['Title']).lower())
    dataset.at[index,'Short Description'] =  re.sub("`|'|!|\||\\|#|\$|%|\^|&|\*|\(|\)|-|_|\[|\]|\{|\}|\"|\'|\+|:|;|<|>|\?|/|`|~|@|=|,|\.", ' ', strip_accents(row['Short Description']).lower())   

for index, row in dataset.iterrows():
    #tokenizing (note as title+description is considered a single document)
    doc_tokens = word_tokenize(row['Title']+" "+row['Short Description']) 
    #removing italian stopwords
    filtered_doc = [w for w in doc_tokens if not w in stop_words_it]
    for word in filtered_doc:
        #stemming tokens
        sword = stemmer.stem(word)
        #Building the dict as basis for the index
        #it is a nested dict like:
        #{term:{doc_id:term_freq}}
        if sword not in term_dict:
            term_dict[sword]= { row['doc_id']: 1 } ##doc_id frequency
        else: 
            if row['doc_id'] not in term_dict[sword]:
                term_dict[sword][row['doc_id']]= 1
            else:
                 term_dict[sword][row['doc_id']] = term_dict[sword][row['doc_id']] + 1    
    

In [6]:
# Writing the inverted index on a file 
# The index stores: Term | idf | posting list with non-normalized frequencies
header = "Term\tidf\tpostings\n"
with open('inverted_index.txt', 'w') as index:
    index.write(header)
    
    for term,postings_dict in term_dict.items():
        line=""
        #sorting by term frequency
        posting_list_sorted = sorted(postings_dict.items(),key=lambda tup: tup[1], reverse=True)
        line += term +"\t"+ str(doc_nr/len(posting_list_sorted))+"\t"+str(posting_list_sorted)+"\n"
        index.write(line)

## Query System
<p>In this part we use the inverted index to retrive the doc_id of the top k documents wrt the inserted query. To retrive the full data of doc_id we will load the data in a Dataframe called <i>announcements</i>, that pratically simulates a database. We will also compute the length of all the document in the dataframe since it will be necessary to the algorithm </p>
<p>If we consider the task of loading the Dataframe <i>announcements</i> and of computing the length of all the documents as an initialization step, done only on query system startup, then the execution of a query can be considered efficient.</p>
<p>Note that the index is kept in memory since it is quite small.</p>





### Startup

In [7]:
sorted_index=pd.read_csv("inverted_index.txt", sep="\t")
#Loading the index in memory, sorted by term, ready to answer to queries
sorted_index.sort_values(by=['Term','idf'], inplace=True,ascending=True)

announcements=pd.read_csv("dataset_rent_rome_kijiji.tsv", sep="\t")
announcements['doc_id']=announcements.index

# Adding length of documents to dataset of announcements
documents_lengths = []
documents_lengths = [None] * (doc_nr + 1)
for index, row in announcements.iterrows():
    doc = row['Title']+" "+row['Short Description']
    documents_lengths[index]= get_document_length(doc)

### System up and running

In [8]:
## Queries processing

query = input("Enter --exit-- to quit the query engine.\n\nInsert Query: ")
while query.lower() != "--exit--":   
    query_vect = process_query(query)
    #print("This is the processed query:\t%s" %str(query_vect))
    
    top_k_doc = perform_query(sorted_index,query_vect,documents_lengths,k=5)
    if not top_k_doc:
        print("Sorry, the query \"%s\" did not produce any result." %query)
    k=0
    for res in top_k_doc:
        #get the elemnt in the dataframe and print them out
        print("\nres:\t%s" %str(res))
        print(announcements.iloc[top_k_doc[k][0]])
        k+=1
    query = input("Enter --exit-- to quit the query engine.\n\nInsert Query: ")


Enter --exit-- to quit the query engine.

Insert Query: casa roma affitto

res:	(915, 0.2524421223153395)
Title                                              Cerco casa affitto 
Short Description    Cerco casa in affitto max 450Â¤ servizi inclus...
Location                                 Acilia / Axa / Casal Palocco 
Price (Euro)                                                      450 
Timestamp                                         24 settembre, 15:16 
Url Adv              https://www.kijiji.it/annunci/affitto/roma-ann...
doc_id                                                             915
Name: 915, dtype: object

res:	(744, 0.1700409947755789)
Title                Cerco: Solo roma cittÃƒâ‚¬, no periferia di ro...
Short Description    Cerco villa in affitto per fare Adorazione Euc...
Location                                                         Roma 
Price (Euro)                                        Contatta l'utente 
Timestamp                                            3 o